In [ ]:
# Adjustable Privacy - train_obfuscator.ipynb
# - Train an obfuscator.
# - Uses Categorical dataset UCI-Adult.
# - Saves models after each epoch number (to google drive and locally).
# - It can stop and resume training.
# - Draws loss, and accuracy plots and saves them (to google drive).
# - Also it can load models and draw plots (from google drive).
# - You can manage notebook parameters in parser block

In [ ]:
# Imports
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
import numpy as np
from collections import OrderedDict
import time
from torch.utils.data import random_split
from math import floor
import torchvision.utils as vutils
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
import itertools
import random
import shutil
from zipfile import ZipFile
import os
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import argparse

In [ ]:
# Parser
parser = argparse.ArgumentParser(description='Adjustable Privacy - Train an obfuscator. '
                                 + 'Uses Categorical dataset UCI-Adult. '
                                 + 'Saves models after each epoch number (to google drive and locally). '
                                 + 'It can stop and resume training.'
                                 + 'Draws loss and accuracy plots and saves them (to google drive and locally). '
                                 + 'Also it can load models and draw plots (from google drive).')

parser.add_argument('--resume', default = False, help = 'Accepts "True" or "False". ')
parser.add_argument('--last_epoch', type=int, default = 0, help = 'In case of resuming training use last saved epoch number and in case of loading a model, set to model number.')
parser.add_argument('--save_path', type=str, required=True, help = 'Full path on your google drive to save model and plots. And also load from it. Like "drive/MyDrive/adjustable-privacy/Models/categorical-Obfuscator/"')
parser.add_argument('--epoch_numbers', type=int, default = 200, help = 'Number of epochs to train model. (when you want load a model, it should set to that model number)')
parser.add_argument('--dataset_path', type=str, default = "", help = 'Full path on your google drive to adult.csv. Like "drive/MyDrive/adjustable-privacy/Datasets/"')

command_string = "--resume False" \
" --last_epoch 0" \
" --save_path drive/MyDrive/adjustable-privacy/Models/categorical-Obfuscator/" \
" --epoch_numbers 200" \
" --dataset_path drive/MyDrive/adjustable-privacy/Datasets/"

args = parser.parse_args(command_string.split())

In [ ]:
# Hyper parameters:
isFirstRun = args.resume=='False'
lastRunEpochNumber = args.last_epoch
manual_seed = 20
learning_rate = 0.001 #0.2
batch_size = 64
files_not_ready = True
dataset_folder_path = args.dataset_path
data_dir = 'adult'
saving_path = args.save_path
# Number of workers for dataloader
workers = 2
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1
# Number of training epochs
num_epochs = args.epoch_numbers

In [ ]:
# Check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
# Mount google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# download dataset and unzip

if files_not_ready:
    dataset_csv_path = dataset_folder_path + '/adult.csv'

    try:
      os.mkdir(data_dir)
      print("data folder created successfully")
    except OSError as e:
      print("Error: %s" % (e.strerror))

    shutil.copyfile(dataset_csv_path, data_dir + r'/adult.csv')

try:
    os.mkdir(saving_path)
    print("saving_path directory created successfully")
except OSError as e:
    print("Error: %s" % (e.strerror))

In [ ]:
#PreProcess dataset:
df = pd.read_csv(os.path.join(data_dir,'adult.csv'))
df = df.replace({'?':np.nan})
df = df.dropna()
df1 = pd.get_dummies(df)
train, test = train_test_split(df1, test_size = 0.2, random_state = 42)
utility_train_true_labels = np.array(train[['income_<=50K','income_>50K']])
utility_test_true_labels = np.array(test[['income_<=50K','income_>50K']])
private_train_true_labels = np.array(train[['gender_Male', 'gender_Female']])
private_test_true_labels = np.array(test[['gender_Male', 'gender_Female']])
x_train = (train.drop(['income_<=50K','income_>50K','gender_Male', 'gender_Female'],axis='columns'))
x_test = (test.drop(['income_<=50K','income_>50K','gender_Male', 'gender_Female'],axis='columns'))
standard_scaler = preprocessing.StandardScaler()
standard_scaler.fit(x_train)
x_train = standard_scaler.transform(x_train)
x_test = standard_scaler.transform(x_test)

In [ ]:
from torch.utils.data import Dataset

class UciAdultDataset(Dataset):
    def __init__(self, X, Y_p, Y_u):
        self.X = X
        self.Y_p = Y_p
        self.Y_u = Y_u
        
    def __len__(self):
        return len(self.Y_p)
    
    def __getitem__(self, idx):
        data = self.X[idx]
        label_p = self.Y_p[idx]
        label_u = self.Y_u[idx]
        data = torch.from_numpy(data)
        label_p = torch.from_numpy(label_p)
        label_u = torch.from_numpy(label_u)
        return data, label_p, label_u

In [ ]:
# Load Datas
train_set = UciAdultDataset(x_train, private_train_true_labels, utility_train_true_labels)
test_set = UciAdultDataset(x_test, private_test_true_labels, utility_test_true_labels)

# DataLoader
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=workers, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=workers, drop_last=True)

In [ ]:
# Decide which device we want to run on
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

In [ ]:
# custom weights initialization
def weights_init(m):
  classname = m.__class__.__name__
  if classname.find('Linear') != -1:
    nn.init.normal_(m.weight.data, 0.0, 0.02)
    nn.init.constant_(m.bias.data, 0)

In [ ]:
# Encoder Model
class Encoder(nn.Module):
    def __init__(self, ngpu):
        super(Encoder, self).__init__()
        self.ngpu = ngpu
        
        # input is 102
        self.fllc1 = nn.Linear(102, 128)
        self.actv1 = nn.ReLU(inplace=True)
        self.fllc2 = nn.Linear(128, 128)
        self.actv2 = nn.ReLU(inplace=True)
        self.fllc4 = nn.Linear(128, 64)
        self.actv4 = nn.ReLU(inplace=True)

        # split features: 128 -> 126 + 2
        # first classifier:
        self.fllc_main_features2 = nn.Linear(64, 32)
        self.actv_main_features2 = nn.ReLU(inplace=True)
        self.fllc_main_features3 = nn.Linear(32, 8)
        self.actv_main_features3 = nn.ReLU(inplace=True)
        self.fllc_main_features4 = nn.Linear(8, 2)
        self.actv_main_features4 = nn.LogSoftmax(dim=1)
        # other features
        self.fllc_other_features1 = nn.Linear(64, 64)
        self.actv_other_features1 = nn.ReLU(inplace=True)
        self.fllc_other_features2 = nn.Linear(64, 62)
        self.actv_other_features2 = nn.ReLU(inplace=True)

    def forward(self, x):
        # Part 1:
        x = self.fllc1(x)
        x = self.actv1(x)
        x = self.fllc2(x)
        x = self.actv2(x)
        x = self.fllc4(x)
        x = self.actv4(x)
        # Part 2:
        # first classifier: 
        y1 = self.fllc_main_features2(x)
        y1 = self.actv_main_features2(y1)
        y1 = self.fllc_main_features3(y1)
        y1 = self.actv_main_features3(y1)
        y1 = self.fllc_main_features4(y1)
        y1 = self.actv_main_features4(y1)
        # other features
        y3 = self.fllc_other_features1(x) 
        y3 = self.actv_other_features1(y3)
        y3 = self.fllc_other_features2(y3) 
        y3 = self.actv_other_features2(y3)
        return y1, y3

In [ ]:
# Decoder Model
class Decoder(nn.Module):
    def __init__(self, ngpu):
        super(Decoder, self).__init__()
        self.ngpu = ngpu
        
        # input size is 64
        self.fllc1 = nn.Linear(64, 128)
        self.actv1 = nn.ReLU(inplace=True)
        self.fllc2 = nn.Linear(128, 128)
        self.actv2 = nn.ReLU(inplace=True)
        self.fllc4 = nn.Linear(128, 102)
        self.actv4 = nn.Sigmoid()

    def forward(self, x):
        x = self.fllc1(x)
        x = self.actv1(x)
        x = self.fllc2(x)
        x = self.actv2(x)
        x = self.fllc4(x)
        x = self.actv4(x)
        return x

In [ ]:
# AE Model
class AEModel(nn.Module):
    def __init__(self, ngpu):
        super(AEModel, self).__init__()
        self.ngpu = ngpu
        self.encoder = Encoder(ngpu).to(device)
        self.decoder = Decoder(ngpu).to(device)

    def forward(self, x):
        y1, y3 = self.encoder(x)
        y = torch.cat((y1, y3), 1)
        x = self.decoder(y)
        return x, y1

In [ ]:
# Create the AE
netAE = AEModel(ngpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    netAE = nn.DataParallel(netAE, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
netAE.apply(weights_init)

In [ ]:
# total parameters
total_params = sum(p.numel() for p in netAE.parameters())
print(f"{total_params:,} total parameters.")

In [ ]:
# Initialize BCELoss function
criterion = nn.MSELoss()
criterion_main_features = nn.NLLLoss()
optimizer = optim.Adam(netAE.parameters(), lr=learning_rate, betas=(beta1, 0.999))

In [ ]:
# Function - Save:
def save_model(name, number, model, res):
  checkpoint = {'res': res,
                'state_dict': model.state_dict()}
  torch.save(checkpoint, saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth')
  return True

In [ ]:
# Function - Load:
def load_model(name, number, model, device):
  
  checkpoint = torch.load(saving_path + 'checkpoint-' + name + '-' + str(number) + '.pth', map_location=device)
  res = checkpoint['res']
  model.load_state_dict(checkpoint['state_dict'])
  return {'model':model,
          'res':res}

In [ ]:
# Save Start Checkpoint
if(isFirstRun):
  res = {'train_losses': [],
         'valid_losses': [],
         'y1_train_losses': [],
         'y1_valid_losses': [],
         'test_mse': [],
         'test_y1_acc': [],
         'epoch_number': 0
        };
  save_model('ae', 0, netAE, res)

In [ ]:
# Load Last Checkpoint:
ae_load = load_model('ae', lastRunEpochNumber, netAE, device)

train_losses = ae_load['res']['train_losses']
valid_losses = ae_load['res']['valid_losses']
y1_train_losses = ae_load['res']['y1_train_losses']
y1_valid_losses = ae_load['res']['y1_valid_losses']
test_mse = ae_load['res']['test_mse']
test_y1_acc = ae_load['res']['test_y1_acc']
last_epoch = ae_load['res']['epoch_number']

In [ ]:
def extract_class_index(labels):
  return labels[:,0]

In [ ]:
# Function - training function
def fit(netAE, train_loader, optimizer, criterion):
    print('Training')
    netAE.train()
    train_loss = 0
    y1_train_loss = 0

    # For each batch in the dataloader
    prog_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, data in prog_bar:
        inputs, labels = data[0], data[2]
        labels = extract_class_index(labels)
        inputs = inputs.to(torch.float32)
        inputs, labels = inputs.to(device), labels.to(device)
        netAE.zero_grad()
        output, y1 = netAE(inputs)
        err = criterion(output, inputs)
        err_y1 = criterion_main_features(y1, labels)
        err.backward(retain_graph=True)
        err_y1.backward(retain_graph=True)
        optimizer.step()
        train_loss += err.item()
        y1_train_loss += err_y1.item()
    train_loss = train_loss / len(train_loader)
    y1_train_loss = y1_train_loss / len(train_loader)
    return train_loss, y1_train_loss

In [ ]:
# Function - validation function
def validate(netAE, valid_loader, optimizer, criterion):
    print('Validating')
    netAE.eval()
    valid_loss = 0
    y1_valid_loss = 0
    # For each batch in the dataloader
    prog_bar = tqdm(enumerate(valid_loader), total=len(valid_loader))
    with torch.no_grad():
        for i, data in prog_bar:
            inputs, labels = data[0], data[2]
            labels = extract_class_index(labels)
            inputs = inputs.to(torch.float32)
            inputs, labels = inputs.to(device), labels.to(device)
            output, y1 = netAE(inputs)
            err = criterion(output, inputs)
            err_y1 = criterion_main_features(y1, labels)
            valid_loss += err.item()
            y1_valid_loss += err_y1.item()
    valid_loss = valid_loss / len(valid_loader)
    y1_valid_loss = y1_valid_loss / len(valid_loader)
    return valid_loss, y1_valid_loss

In [ ]:
# Calc Accuracy
def calcAccuracyTest(netAE, test_loader):
    print('Testing')
    netAE.to(device)
    print("Calculating Accuracy...")
    netAE.eval()
    mse_loss = 0
    y1_accuracy = 0
    prog_bar = tqdm(enumerate(test_loader), total=len(test_loader))
    with torch.no_grad():
        for i, data in prog_bar:
            inputs, labels = data[0], data[2]
            labels = extract_class_index(labels)
            inputs = inputs.to(torch.float32)
            inputs, labels = inputs.to(device), labels.to(device)
            output, y1 = netAE(inputs)
            mse = criterion(output, inputs)
            ps_y1 = torch.exp(y1)
            top_p_y1, top_class_y1 = ps_y1.topk(1, dim=1)
            equals_y1 = top_class_y1 == labels.view(*top_class_y1.shape)
            acc_y1 = equals_y1.sum().item()
            mse_loss += mse.item()
            y1_accuracy += (acc_y1 / len(equals_y1))
    mse_loss = mse_loss / len(test_loader)
    y1_accuracy = y1_accuracy / len(test_loader)
    return mse_loss, y1_accuracy

In [ ]:
# Training Loop
netAE.to(device)
save_every_epoch = 1

start = time.time()
print("Starting Training Loop...")

for epoch in range(last_epoch+1, num_epochs+1):
    print(f"Epoch {epoch}/{num_epochs}: ")
    train_loss, y1_train_loss = fit(netAE, train_loader, optimizer, criterion)
    valid_loss, y1_valid_loss = validate(netAE, test_loader, optimizer, criterion)
    mse_loss, y1_accuracy = calcAccuracyTest(netAE, test_loader)

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    y1_train_losses.append(y1_train_loss)
    y1_valid_losses.append(y1_valid_loss)
    test_mse.append(mse_loss)
    test_y1_acc.append(y1_accuracy)

    res = {'train_losses': train_losses,
           'valid_losses': valid_losses,
           'y1_train_losses': y1_train_losses,
           'y1_valid_losses': y1_valid_losses,
           'test_mse': test_mse,
           'test_y1_acc': test_y1_acc,
           'epoch_number': epoch
          };
    if epoch % save_every_epoch == 0:
      save_model('ae', epoch, netAE, res)


    print(f"Train Loss: {train_loss:.6f}")
    print(f"Valid Loss: {valid_loss:.6f}")
    print(f"Main Label Train Loss: {y1_train_loss:.6f}")
    print(f"Main Label Valid Loss: {y1_valid_loss:.6f}")

end = time.time()
print(f"Training time: {(end-start)/60:.3f} minutes")

print('TRAINING COMPLETE')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print('Loss plot...')

# loss plots
plt.figure(figsize=(10,7))
plt.title("Train-Valid Loss Trend")
plt.plot(train_losses, color='green', label='Training Loss')
plt.plot(test_mse, color='red', label='Validation Loss')
plt.legend(frameon=False)
plt.xlabel("epochs")
plt.ylabel("Loss")
plt.savefig(saving_path + "loss_plot.png")
plt.show()